<a href="https://colab.research.google.com/github/Anaalicerg20/day-night-classificaton/blob/main/RAIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problema 1: classificação dia/noite**

**Abordagem 1 — Análise da Iluminação por Histograma em Tons de Cinza**

A primeira abordagem, considerada a mais eficiente, baseia-se na análise da iluminação da imagem por meio do histograma em tons de cinza. Inicialmente, as imagens são convertidas para escala de cinza, representando apenas a luminância, e em seguida é gerado o histograma de intensidades.

A análise dos histogramas mostrou que imagens diurnas apresentam picos distribuídos ao longo de diversas faixas de intensidade, indicando maior variedade de brilhos. Além disso, essas imagens normalmente apresentam pelo menos um pico relevante em intensidades acima de 127(aproximadamente a metade das intensidades do histograma), o que reflete a presença de regiões bem iluminadas, como céu, nuvens, fachadas claras ou superfícies refletindo luz solar. Já imagens noturnas, mesmo com iluminação artificial, concentram a maior parte dos picos em baixas intensidades, geralmente abaixo de 50, com poucos picos intermediários entre 50 e 127, devido ao predomínio de áreas escuras.

Esse comportamento permitiu uma separação eficaz entre dia e noite, tornando essa abordagem a principal. Contudo, o método pode falhar em imagens noturnas extremamente claras, especialmente quando a iluminação artificial é intensa e capaz de clarear grande parte da cena, fazendo o histograma se assemelhar ao de imagens diurnas.

**Abordagem 2 — Análise Cromática da Iluminação Artificial**

A segunda abordagem baseia-se na predominância de cores quentes, especialmente tons alaranjados e avermelhados, típicos de iluminação artificial noturna. Para isso, são analisadas razões entre os canais cromáticos, comparando a intensidade média dos canais vermelho e verde em relação ao canal azul.

A razão
(R + G)/B mede o quanto os tons quentes dominam a cena em relação aos tons frios. A condição (R+G)/B > 2 indica que a soma das intensidades quentes é pelo menos o dobro da intensidade do canal azul, caracterizando uma dominância significativa de iluminação quente. Adicionalmente, a razão R/B > 1 garante que o canal vermelho participa dessa dominância, evitando classificações incorretas em cenas predominantemente verdes ou cromaticamente neutras.

Imagens que satisfazem simultaneamente essas duas condições são classificadas como noturnas, pois refletem uma assinatura cromática compatível com fontes de luz artificial urbana.

Embora simples, essa abordagem é mais suscetível a erros quando utilizada isoladamente. Ela pode falhar em noites sem iluminação urbana, em cenas noturnas iluminadas por luz branca ou fria, bem como em imagens de nascer e pôr do sol, que naturalmente apresentam tons quentes apesar de ocorrerem durante o dia.

Por esse motivo, essa abordagem não é adequada como critério único, devendo ser utilizada de forma complementar à análise baseada na iluminação global e na distribuição dos níveis de intensidade da imagem.

**Uso Complementar das Abordagens**

A combinação das duas abordagens permite lidar com casos ambíguos. A classificação deve ser baseada principalmente na distribuição de intensidades do histograma em tons de cinza. Em situações em que a imagem é predominantemente escura, mas apresenta picos em intensidades acima de 127, a análise cromática pode ser utilizada para verificar se esses picos correspondem a iluminação artificial quente, caracterizando uma noite urbana, e não um dia nublado.

In [ ]:
#Para que o código funcione, você deve baixar a pasta Dataset RAIA e carregá-la
#diretamente na raiz do seu Google Drive para que o caminho permaneça
#/content/drive/MyDrive/Dataset RAIA/.

import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive

#Google Drive
drive.mount('/content/drive')
caminho_fotos = '/content/drive/MyDrive/Dataset RAIA/'

#Abordagem 1:  Histograma: dispersão de luminância
def analise_abordagem_1(image):
    #transforma a img pra cinza e calcula seu historiograma
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([img_gray], [0], None, [256], [0, 256])
    hist = hist.flatten().astype(np.float64)
    hist_norm = hist / (hist.sum() + 1e-6)

    # Suavização
    kernel = np.array([1, 1, 1], dtype=np.float64) / 3
    hist_smooth = np.convolve(hist_norm, kernel, mode="same")

    min_height = 0.002
    tem_pico = False
    # Procura picos na metade clara do histograma
    for i in range(127, 255):
        if (hist_smooth[i] > hist_smooth[i-1] and
            hist_smooth[i] > hist_smooth[i+1] and
            hist_smooth[i] > min_height):
            tem_pico = True
            break

    classificacao =  "dia" if tem_pico else "noite"

    return classificacao, img_gray, hist

import cv2
import numpy as np

#Abordagem 2: temperatura das cores
def analise_abordagem_2(image):
    # separa os canais de cor
    B = image[:, :, 0].astype(float)
    G = image[:, :, 1].astype(float)
    R = image[:, :, 2].astype(float)

    mean_B, mean_G, mean_R = B.mean(), G.mean(), R.mean()
    eps = 1e-6

    # razões entre os canais
    rg_over_b = (mean_R + mean_G) / (mean_B + eps)
    r_over_b  = mean_R / (mean_B + eps)

    #imagem: quente > fria = noite(luzesa artificiais)
    if rg_over_b > 2 and r_over_b > 1:
        classificacao = "noite"
    else:
        classificacao = "dia"

    return classificacao, mean_R, mean_G, mean_B, rg_over_b, r_over_b


#Visualizar resultado das abordagens
def mostrar_imagem(nome):
    caminho_completo = os.path.join(caminho_fotos, nome)
    img = cv2.imread(caminho_completo)

    if img is None:
        print(f"Imagem '{nome}' não encontrada em {caminho_fotos}")
        return

    # Abordagem 1
    res1, img_gray_vis, hist_vis = analise_abordagem_1(img)

    # Abordagem 2
    res2, mean_R, mean_G, mean_B, rg_over_b, r_over_b = analise_abordagem_2(img)

    # Plot
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(16, 4))

    #Original
    plt.subplot(1, 3, 1)
    plt.title(f"Original - {nome}")
    plt.imshow(img_rgb)
    plt.axis("off")

    # Cinza
    plt.subplot(1, 3, 2)
    plt.title("Cinza")
    plt.imshow(img_gray_vis, cmap="gray")
    plt.axis("off")

    # 3) Histograma
    plt.subplot(1, 3, 3)
    plt.title("Histograma – níveis de cinza")
    plt.xlabel("Intensidade (0 = preto, 255 = branco)")
    plt.ylabel("Quantidade de pixels")
    plt.plot(hist_vis)
    plt.xlim([0, 255])

    texto = (
        f"Abordagem1: {res1} | Abordagem2: {res2}\n"
        f"Média R: {mean_R:.2f} | Média G: {mean_G:.2f} | Média B: {mean_B:.2f} | "
        f"(R+G)/B: {rg_over_b:.2f} | R/B: {r_over_b:.2f}"
    )
    plt.figtext(0.5, -0.15, texto, ha="center", fontsize=10)

    plt.tight_layout()
    plt.show()


#Tabela
lista_abordagem1 = []
lista_abordagem2 = []
lista_veredito = []

nomes_arquivos = [f'foto{i}.jpg' for i in range(1, 31)]

print("Iniciando processamento das 30 imagens...")

for nome in nomes_arquivos:
    caminho_completo = os.path.join(caminho_fotos, nome)
    img = cv2.imread(caminho_completo)

    if img is not None:
        res1, _, _ = analise_abordagem_1(img)

        res2, *_ = analise_abordagem_2(img)

        veredito = "convergente" if res1 == res2 else "divergente"

        lista_abordagem1.append(res1)
        lista_abordagem2.append(res2)
        lista_veredito.append(veredito)
    else:
        lista_abordagem1.append("erro")
        lista_abordagem2.append("erro")
        lista_veredito.append("não encontrado")

df = pd.DataFrame({
    'Imagem': nomes_arquivos,
    'Abordagem1 (Histograma)': lista_abordagem1,
    'Abordagem2 (Cores)': lista_abordagem2,
    'Resultado_Final': lista_veredito
})

display(df)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Iniciando processamento das 30 imagens...


,Imagem,Abordagem1 (Histograma),Abordagem2 (Cores),Resultado_Final
0,foto1.jpg,dia,dia,convergente
1,foto2.jpg,dia,dia,convergente
2,foto3.jpg,noite,dia,divergente
3,foto4.jpg,dia,dia,convergente
4,foto5.jpg,noite,dia,divergente
5,foto6.jpg,dia,dia,convergente
6,foto7.jpg,noite,noite,convergente
7,foto8.jpg,dia,dia,convergente
8,foto9.jpg,dia,noite,divergente
9,foto10.jpg,noite,dia,divergente


In [ ]:
#caso queira ver alguma imagem com seu historiograma e dados: rode o
#código e selecione as imagen desejadas.

while True:
    resp = input("\nQuer ver alguma imagem com cinza + histograma + métricas? (s/n): ").strip().lower()

    if resp == "n":
        print("Ok! Encerrado.")
        break

    elif resp == "s":
        quais = input(
            "Digite os nomes (ex: foto3.jpg,foto10.jpg) "
            "ou números (ex: 3,10): "
        ).strip()

        selecionadas = []
        partes = [p.strip() for p in quais.split(",") if p.strip()]

        for p in partes:
            if p.isdigit():
                selecionadas.append(f"foto{int(p)}.jpg")
            else:
                selecionadas.append(p)

        for nome in selecionadas:
            mostrar_imagem(nome)

    else:
        print("Resposta inválida. Digite 's' para sim ou 'n' para não.")


Quer ver alguma imagem com cinza + histograma + métricas? (s/n): n
Ok! Encerrado.
